# Search Engine using the BM25 Algorithm

The goal is to apply one method (BM25) of ranking documents according to a query.

In [45]:
# imports
from nltk.corpus import reuters
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import time
import random

First we need a database to use as an example. In a search engine like google these documents would be web pages. In our case I will use the reuters corpus from the nltk package. This corpus consists in over 10k news over various topics.

In [28]:
# creating dataframe of documents and ids
corpus = [{"id": idu, "content": " ".join(reuters.raw(idu).split())} for idu in reuters.fileids()]
corpus = pd.DataFrame(corpus)
print(corpus.shape)
corpus.head(10)

(10788, 2)


,content,id
0,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,test/14826
1,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,test/14828
2,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,test/14829
3,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER Tha...,test/14832
4,INDONESIA SEES CPO PRICE RISING SHARPLY Indone...,test/14833
5,AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS...,test/14839
6,INDONESIAN COMMODITY EXCHANGE MAY EXPAND The I...,test/14840
7,SRI LANKA GETS USDA APPROVAL FOR WHEAT PRICE F...,test/14841
8,WESTERN MINING TO OPEN NEW GOLD MINE IN AUSTRA...,test/14842
9,SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERG...,test/14843


In order to preprocess our data to speed the search we will build the Document Term Matrix. This matrix has its columns being the documents, the rows being its individual words and the values are counts of how many times the word appears in the document.

In [41]:
# selecting data
docs = corpus['content'].values
names = corpus['id'].values
# building matrix
vectorizer = CountVectorizer()
x1 = vectorizer.fit_transform(docs)
term_matrix = pd.DataFrame(x1.toarray().transpose(), index=vectorizer.get_feature_names())
term_matrix.columns = names
print(term_matrix.shape)
term_matrix.iloc[3260:3270, 4:12]

(30916, 10788)


,test/14833,test/14839,test/14840,test/14841,test/14842,test/14843,test/14844,test/14849
arkadi,0,0,0,0,0,0,0,0
arkansas,0,0,0,0,0,0,0,0
arkla,0,0,0,0,0,0,0,0
arl,0,0,0,0,0,0,0,0
arlan,0,0,0,0,0,0,0,0
arland,0,0,0,0,0,0,0,0
arm,0,0,0,0,0,0,0,0
armacost,0,0,0,0,0,0,0,0
armand,0,0,0,0,0,0,0,0
armco,0,0,0,0,0,0,0,0


To execute the actual search we will use an algorithm called [BM25](https://en.wikipedia.org/wiki/Okapi_BM25 "algorithm wikipedis's page"). The function that makes the search is the follow:

In [54]:
def search(query, corpus2=corpus.copy()):
    # algorithm parameters
    k = 1.5
    b = 0.75
    # calculating search time
    t00 = time.time()
    # splittng query into words
    query_words = query.split()
    # selecting rows from doc term matrix only for query words
    freq_word_doc = term_matrix.loc[query_words].dropna()
    freq_m = np.matrix(freq_word_doc)
    # number of documents containing word qi
    n_wordi = freq_word_doc.apply(lambda x: sum(x > 0), axis=1)
    # number of words for each document
    len_doc = term_matrix.apply(lambda x: sum(x), axis=0)
    # average document lenght in the collection
    avg_len_doc = sum(len_doc)/len(len_doc)
    # number of documents in the collection
    num_doc = term_matrix.shape[1]
    # IDFs for each word
    idfs = n_wordi.apply(lambda x: math.log((num_doc - x + 0.5) / (x + 0.5)))
    idfs_m = np.matrix(idfs)
    len_doc_m = np.matrix([len_doc for i in range(len(idfs))])
    # calculating scores using bm25
    lens_m = k * (1 - b + (b * len_doc_m / avg_len_doc))
    div = (freq_m * (k + 1)) / (freq_m + lens_m)
    fin = idfs_m * div
    # inserting score column in the dataframe
    for it in range(len(names)):
        corpus2.loc[corpus2['id'] == names[it], 'score'] = fin.tolist()[0][it]
    # sorting documents by score
    corpus2.sort_values(by='score', ascending=False, inplace=True)
    print('search time: {} s'.format(str(time.time() - t00)))
    return corpus2

With the search function built and our documents corpus ready we can execute a search. To test its quality We will select a document randomly from the corpus:

In [49]:
doc_number = random.randint(0,corpus.shape[0])
print(corpus.loc[doc_number]['id'])
print()
print(corpus.loc[doc_number]['content'])

training/3890

HEINZ &lt;HNZ> HAS HIGHER NET DESPITE HIGHER COSTS H.J. Heinz Co said net income for the third quarter rose 18.2 pct despite an 17.2 pct increase in marketing expenses. Meanwhile, the company said it raised its quarterly dividend to 28 cts a share from 25 cts a share in part on the expectation that its tax rate under the new tax law will result in greater cash flow. For the third quarter ended January 31, Heinz earned 74.7 mln dlrs, or 55 cts a share, up from earnings of 63.2 mln dlrs, or 46 cts a share, for the year-ago quarter. For the nine months, the company posted a profit of 244.4 mln dlrs, or 1.78 dlrs a share, compared with a profit of 219.7 mln dlrs, or 1.60 dlrs a share, for the year-ago period. "Based on the company's performance for the first nine months, we expect to achieve our 23rd consecutive year of new growth records," Anthony J.F. O'Reilly, Heinz's newly elected chairman.


Now we build a search query to find this document:

In [50]:
query = 'heinz net income and dividends'

Executing the search:

In [55]:
res = search(query=query)
print()
print(res.head())

search time: 58.41211223602295 s

                                                content              id  \
6845  HEINZ &lt;HNZ> HAS HIGHER NET DESPITE HIGHER C...   training/3890   
8350  H.J. HEINZ &lt;HNZ> POISED FOR RECORD YEAR H.J...   training/6215   
3544  CLABIR &lt;CLG> DIVIDENDS NOT TAXABLE Clabir C...  training/10817   
6819  H.J. HEINZ RAISES QUARTERLY TO 28 CTS FROM 25 ...   training/3851   
6818  H.J. HEINZ CO 3RD QTR SHR 55 CTS VS 46 CTS H.J...   training/3850   

          score  
6845  15.382090  
8350  14.351927  
3544  13.754047  
6819  12.674172  
6818  12.586586  


We can see the first result is exactly our chosen news article